To-do:

1. create single symmetrical co-occurrence matrix (with general semantic embeddings)
2. determine target/landmark pairings from this matrix 
3. visualize object clusters
4. compare clusters across language models
5. design room

In [4]:
import numpy as np
import pandas as pd
import gzip
import fastparquet
import matplotlib.pyplot as plt
import tensorflow as tf
import gensim.downloader as api

In [5]:
model = api.load('conceptnet-numberbatch-17-06-300')

[==================================================] 100.0% 1168.7/1168.7MB downloaded


In [15]:
# read conceptnet
conceptnet = pd.read_parquet('conceptnet_small.parquet.gzip',columns=['uri','relation','start_node','end_node','info'])

# choose room for which to search relations
room =  'kitchen'
cn_room = '/c/en/' + room
node = 'end_node'
object_relations = conceptnet[conceptnet[node] == cn_room]

# create list of objects typically associated with chosen room type
objs = []

for i in range(object_relations.shape[0]):
    objs.append(object_relations['start_node'].iloc[i])

In [3]:
# finalize object list
all_objs = pd.read_excel('post_objects.xlsx')

In [8]:
#create cooccurrence matrix 
cooccur_target = {} 

for i in range(len(landmark)):
    
    selected_targets = list(pair['target'][pair['landmark']==landmarks[i]])
    cooccur = np.zeros((len(selected_targets),len(selected_targets)))
    
    inputs = tokenizer(target, return_tensors="np")
    idx_selected_targets = []

    for x in range(len(selected_targets)):
        idx_selected_targets.append(inputs.input_ids[x][1])

    for j in range(len(selected_targets)):
    
        sentence = 'The [MASK] should be in the ' + room + ' next to the ' + selected_targets[j] + ' on the ' + landmark[i] + '.'

        inputs = tokenizer(sentence, return_tensors="tf")
        logits = model(**inputs).logits

        mask_token_index = tf.where((inputs.input_ids == tokenizer.mask_token_id)[0])
        selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

        cooccur[j,:] = tf.nn.softmax(selected_logits.numpy()[0,idx_selected_targets])


In [9]:
# cluster corrrelation matrix and visualize
cooccur = np.zeros((len(landmark),len(target)))

for i in range(len(landmark)):
    for j in range(len(target)):
        sentence = 'The [MASK] is on the ' + landmark[i] + ' in the ' + room + '.'
        result = unmasker(sentence, targets = target[j])
        cooccur[i,j] = result[0].get("score")

In [11]:
# determine target landmark pairings
num_targets = []

for i in range(len(landmarks)):
    num_targets.append(sum(pair.landmark == landmarks[i]))
    
print(num_targets)

[9, 2, 15, 13, 4, 9]


In [12]:
pair['target'][pair['landmark']==landmarks[5]]

11          crab
15           fan
23    ingredient
24          iron
25        kettle
27         maker
33           oil
36           pan
42           pot
Name: target, dtype: object

In [13]:
# design room

0    counter
1      floor
2      table
3      shelf
4       wall
5      stove
Name: landmark, dtype: object